In [21]:
# Import libraries
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import IsolationForest

# Load the json collection as a list of dictionaries
with open("C:/Users/John/Downloads/mock_orders.json", "r") as f:
    json_collection = json.load(f)

# Extract the keys and values from each json object
keys = []
values = []
for obj in json_collection:
    keys.append(list(obj.keys()))
    values.append(list(obj.values()))

# Convert all values to strings
for i in range(len(keys)):
    for j in range(len(keys[i])):
        keys[i][j] = str(keys[i][j])
for i in range(len(values)):
    for j in range(len(values[i])):
        values[i][j] = str(values[i][j])

# Create a label encoder for keys and values
label_encoder = LabelEncoder()

# Flatten the keys and values into one-dimensional arrays
keys = np.ravel(keys)
values = np.ravel(values)

# Fit and transform the keys and values into integer labels
keys_encoded = label_encoder.fit_transform(keys)
values_encoded = label_encoder.fit_transform(values)

# Create a one-hot encoder for keys and values
one_hot_encoder = OneHotEncoder(handle_unknown="ignore")

# Stack the keys_encoded and values_encoded arrays vertically
keys_values = np.vstack((keys_encoded, values_encoded))

# Fit the one-hot encoder on the 2D array
one_hot_encoder.fit(keys_values)

# Transform the integer labels into binary features
keys_encoded = keys_encoded.reshape(-1,1)
keys_one_hot = one_hot_encoder.transform(keys_encoded)
values_one_hot = one_hot_encoder.transform(values_encoded)

# Concatenate the one-hot encoded keys and values as features
features = np.concatenate((keys_one_hot, values_one_hot), axis=2)

# Train an isolation forest model to detect outliers
model = IsolationForest(random_state=0)
model.fit(features)

# Predict the anomaly scores for each json object
scores = model.score_samples(features)

# # Print the json objects with negative scores (anomalies)
# for i, score in enumerate(scores):
#     if score < 0:
#         print(f"Anomaly detected: {json_collection[i]}")

# Plot the scores on a graph
x = np.linalg.norm(features, axis=1) # compute the norm of each feature vector
plt.plot(x, scores, color='blue', label='Scores') # create a line plot of scores
plt.scatter(x, scores, c=scores < 0, cmap='coolwarm', label='Anomalies') # create a scatter plot of anomalies with red color
plt.xlabel('Feature Vector Value') # add x-axis label
plt.ylabel('Score')

TypeError: __init__() got an unexpected keyword argument 'categorical_features'